###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2020 Adam Wickenheiser

## Lab 1 Simulator

This Python code is a simulation of the mass-spring-damper experiment used in Lab 1.  This code will output a text file of data in the same format as output by the data acquisition software used in lab.

The following cells only need to be run once to load libraries and define functions.  We define the function `cart_free_response`, to set up the given physical characteristics of the system as well as the equation of motion to govern the system.  This function is called by `solve_ivp` in a later cell, which is similar to MATLAB's `ode23` or `ode45`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import matplotlib.patches as mpatches
from matplotlib.transforms import Affine2D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
def cart_free_response(t,z,n_weights,damp_percent):
    
    #n_weights = 2
    #damp_percent = 0.5
    m = 0.3 + 0.49*n_weights
    b = 4 + 10*damp_percent
    k = 660.0
    mu = 0.02
    g = 9.81
    zdot = [z[1], (b*z[1]+k*z[0]+mu*m*g*np.sign(z[1]))/(-m)]
    
    return zdot

In the following cell, set the parameters of the experiment.  These are:

1. The initial displacement of the cart in centimeters
1. The number of weights attached to the cart (0-3)
1. The tightness of the air damper valve screw (0-1)
1. Duration of the simulation in seconds

Each time you change any of these parameters, you must re-run the cell.

In [ ]:
x_init = 0.03        # initial cart displacement [cm]
n_weights = 2        # number of weights attached to the cart
damp_percent = 0.5   # tightness of the air damper valve screw
tf = 3.0             # duration of the simulation [s]

Run the following cell to run the simulation, plot the position vs. time graph, and create an animation of the cart's motion.  Note: this may take a few seconds to run; make sure you can observe the animation and plot before proceeding.

In [ ]:
t_span = [0.0, tf]
dt = 0.009
t_eval = np.arange(t_span[0],t_span[1],dt)
z0 = [x_init, 0]
sol = solve_ivp(cart_free_response,t_span,z0,t_eval=t_eval,args=(n_weights,damp_percent))

plt.figure(figsize=(10,5))
plt.plot(sol.t,sol.y[0,:])
plt.ylabel('Cart position [cm]')
plt.xlabel('Time [s]')

fig, ax = plt.subplots() # You may need to adjust the figsize to fit your screen

# Draw the cart
w_cart = 12.7    # cart width [cm]
h_cart = 5.4     # cart height [cm]
d_wheel = 2.5    # wheel diameter [cm]

cart = mpatches.Rectangle([-w_cart/2,d_wheel],w_cart,h_cart,linewidth=1,edgecolor='k',facecolor='b')
ax.add_patch(cart)
left_wheel = mpatches.Circle((-w_cart/4,d_wheel/2),d_wheel/2,linewidth=1,edgecolor='k',facecolor='w')
ax.add_patch(left_wheel)
right_wheel = mpatches.Circle((w_cart/4,d_wheel/2),d_wheel/2,linewidth=1,edgecolor='k',facecolor='w')
ax.add_patch(right_wheel)
motor_input = mpatches.Rectangle([-15.1,d_wheel+h_cart/4],0.1,h_cart/2,linewidth=1,edgecolor='k',facecolor='k')
ax.add_patch(motor_input)
x_spring = np.linspace(0,1,501)
y_spring = np.sin((2*np.pi*10)*x_spring)
spring = plt.plot(x_spring,y_spring,color='k',linewidth=1)[0]
ax.add_line(spring)
plt.xlim((-16,16))
plt.ylim((0,12))

# Initialize the axes for animation
def init():
    ax.set_aspect('equal')
    return (cart,left_wheel,right_wheel,motor_input,spring)

    
# Update the position of the cart at time t_eval[i]
def update(i):
    x_cart = 100*sol.y[0,i]
    
    cart.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    left_wheel.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    right_wheel.set_transform(Affine2D().translate(x_cart,0) + ax.transData)
    spring.set_transform(Affine2D().scale(15-w_cart/2+x_cart,1) + Affine2D().translate(-15,d_wheel+h_cart/2) + ax.transData)
    return (cart,left_wheel,right_wheel,motor_input,spring)

ani = FuncAnimation(fig, update, frames=range(int(t_eval.size)), init_func=init, interval=dt, blit=True, repeat=False)
plt.close()
HTML(ani.to_jshtml(1/dt))

Run the following cell to save your data in a text file in the current folder.  This file will contain time and position ("Encoder 1 Pos") data points.

In [ ]:
dataout = np.zeros((t_eval.size,6))
dataout[:,0] = range(t_eval.size)
dataout[:,1] = sol.t
dataout[:,3] = 160410*sol.y[0,:]

print('Enter file name for saving data (without extension):')
filename = input()+'.txt'
np.savetxt(filename,dataout,fmt=['%d','%2.3f','%d','%d','%d','%d'],delimiter='\t',header=' Sample   Time     Commanded Pos   Encoder 1 Pos   Encoder 2 Pos   Encoder 3 Pos \n')

data2 = np.genfromtxt('testout.txt',comments=';',skip_header=3,skip_footer=1)
t = data2[:,1]     # time is column 1
x = data2[:,3]     # position is column 3 (recall column numbering starts at 0)
plt.plot(t,x);
print(t[0])
print(x[0])

Now, you have a data file that is identical to the in-lab hardware you can use to analyze the system.  You should repeat the simulation for the other trials by updating the parameters in the third cell from the top and running the subsequent cells.